In [ ]:
import os
import logging
import openai
import pandas as pd
import time
from tqdm import tqdm
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

@retry(wait=wait_random_exponential(min=2, max=60), stop=stop_after_attempt(6))
def make_completion_request(model_name, prompt, question):
    # start_sequence = "\nA:"
    # restart_sequence = "\n\nQ: "

    input = prompt.replace("{question}", question)
    # logging.info(f"input: {input}")

    response = openai.Completion.create(
        model=model_name,
        prompt=input,
        temperature=0,
        max_tokens=50,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["\n"],
    )

    return response


def get_davinci_completions(model_name, data, run_name, prompt, batch_size):
    openai.api_key = os.getenv("OPENAI_API_KEY")
    job_df = {"q_id": [], "question": [], "answer": []}

    # print(data.head())
    for index in tqdm(range(0, len(data))):
        question = data[index]

        try:
            if index % 50 == 0 and index != 0:
                print("************", index)
                time.sleep(10)
            response = make_completion_request(model_name, prompt, question)

            if response.choices[0].text.strip() == "":
                job_df["answer"].append("<no response>")
            else:
                job_df["answer"].append(response.choices[0].text.strip())
        except Exception as e:
            print("Could not get response. Here is the exception:", str(e))
            job_df["answer"].append("<exception>")

        job_df["q_id"].append(index)
        job_df["question"].append(question)
        # logging.info("Got response: ", response)
        # logging.info("\n\n\n")
        
        final_df = pd.DataFrame(job_df)
        final_df.to_csv(f"{run_name}_predictions.csv", index=False, sep="\t")
    return final_df


def get_davinci_completions_mp(model_name, data, run_name, prompt, batch_size):
    openai.api_key = os.getenv("OPENAI_API_KEY")
    job_df = {"q_id": [], "question": [], "answer": []}

    # chunk df into batches of size 100
    # for each batch, create a new process
    

    # print(data.head())
    for index in tqdm(range(0, len(data))):
        question = data[index]

        
    return final_df

def handle_qa_pair(index, question, model_name, prompt, run_name, job_df):
    try:
        if index % 50 == 0 and index != 0:
            print("************", index)
            time.sleep(10)
        response = make_completion_request(model_name, prompt, question)

        if response.choices[0].text.strip() == "":
            job_df["answer"].append("<no response>")
        else:
            job_df["answer"].append(response.choices[0].text.strip())
    except Exception as e:
        print("Could not get response. Here is the exception:", str(e))
        job_df["answer"].append("<exception>")

    job_df["q_id"].append(index)
    job_df["question"].append(question)
    # logging.info("Got response: ", response)
    # logging.info("\n\n\n")
    
    final_df = pd.DataFrame(job_df)
    final_df.to_csv(f"{run_name}_predictions.csv", index=False, sep="\t")


In [2]:
import pandas as pd

def load_dataset(ds_path):
    try:
        data = pd.read_csv(ds_path)
        data["Question"]
    except:
        data = pd.read_csv(ds_path, delimiter="\t")

    return data

In [3]:
import random
import time

def make_completion_request(model_name, prompt, question, index):
    delay = random.randint(100, 1000)
    
    # Wait for the specified delay
    # time.sleep(delay / 1000)
    time.sleep(index)

    return "this is an answer"

In [5]:
import multiprocessing as mp
from tqdm import tqdm

# ele = {"qid": 0, "question": "q", "answer": "a"}

def handle_qa_pair(index, question, model_name, prompt, q):
    print("Handling qa pair: ", index)
    try:
        # if index % 50 == 0 and index != 0:
        #     print("************", index)
        #     time.sleep(10)
        response = make_completion_request(model_name, prompt, question, index)

        # if response.choices[0].text.strip() == "":
        #     answer = "<no response>"
        # else:
        #     answer = response.choices[0].text.strip()
        answer = response
    except Exception as e:
        print("Could not get response. Here is the exception:", str(e))
        answer = "<exception>"

    # Put response into queue
    q.put({"qid": index, "question": question, "answer": answer})
    print("Finished job: ", index)
    # print("\n")

# def worker_function(item, q):
#     """
#     do some work, put results in queue
#     """
#     res = f'item: {item} - result: {item ** 2}'
#     print(res)
#     q.put(res)


def listener(q):
    """
    continue to listen for messages on the queue and writes to file when receive one
    if it receives a '#done#' message it will exit
    """
    with open('output_new.txt', 'wb') as f:
        while True:
            m = q.get()
            print("Heres the message: ", m)
            if m == '#done#':
                print("Got break")
                break
            f.write(f"{m['qid']}\t{m['question']}\t{m['anwser']}\n")
            print("Wrote line to file")
            f.flush()


manager = mp.Manager()
q = manager.Queue()
file_pool = mp.Pool(1)
file_pool.apply_async(listener, (q, ))

pool = mp.Pool(5)
jobs = []

data = load_dataset("/home/wallat/temporal-llms/data/Event-focused Questions/Explicitly Time-Scoped Questions.csv")
# print(data.head())

for index, row in data[:30].iterrows():
    question = row["Question"]
    job = pool.apply_async(handle_qa_pair, (index, question, "<model_name>", "<prompt>", q))
    jobs.append(job)

for job in tqdm(jobs):
    job.get()

q.put('#done#')  # all workers are done, we close the output file
pool.close()
pool.join()

  0%|          | 0/30 [00:00<?, ?it/s]

Handling qa pair: Handling qa pair:  Handling qa pair: Handling qa pair: Handling qa pair: 1   
2 03
4


Finished job:  0
Heres the message:  {'qid': 0, 'question': 'Which two U.S. States had a border dispute that had to be settled by the U.S. Supreme Court in April of 2001?', 'answer': 'this is an answer'}Handling qa pair: 
 5
Finished job:  1


  7%|▋         | 2/30 [00:01<00:14,  1.96it/s]

Handling qa pair:  6
Finished job:  2


 10%|█         | 3/30 [00:02<00:19,  1.40it/s]

...

 93%|█████████▎| 28/30 [01:27<00:10,  5.44s/it]

Finished job:  28


 97%|█████████▋| 29/30 [01:33<00:05,  5.61s/it]

Finished job:  29


100%|██████████| 30/30 [01:39<00:00,  3.32s/it]
